In [1]:
from google.colab import drive 
drive.mount("/content/gdrive", force_remount=True)

Mounted at /content/gdrive


In [15]:
import os
os.getcwd()
# Change directory for nicer imports
%cd "/content/gdrive/My Drive/Data_Mining_Cup/05 Code/"

import pandas as pd
import numpy as np
# from matplotlib import pyplot as plt 
from os import listdir
from os.path import isfile, join

/content/gdrive/My Drive/Data_Mining_Cup/05 Code


## Read the files in the "final_submissions" folder

In [16]:
submission_path_read = "./final_submissions"
subfiles = list(filter(lambda x: x[-4:] == ".csv", [f for f in listdir(submission_path_read ) if isfile(join(submission_path_read , f))]))

submission_path_save = "./final_submissions_DMC"

In [17]:
print(subfiles)

['FINAL_submission_Naive_XGBoost_Model_None.csv', 'test_submission_TEST_Naive_XGBoost_Model_k_promo.csv', 'test_submission_non_recursive_28062020.csv', 'submission_XGBoost_final_k_promo_final.csv', 'submission_XGBoost_final.csv', 'submission_XGBoost_final_k_promo_final_corrected.csv', 'submission_XGBoost_final_k_promo_final_corrected_handcrafted.csv']


In [18]:
submissions = {}
# submissions_monet_val = {}
# submissions_analysis ={}

_loaded = 0
for f in subfiles:
  print(f"> Load file {f}")
  try:
    submissions[f[:-4]] = pd.read_csv(f'{submission_path_read}/{f}',delimiter='|', index_col="itemID")
    assert not((submissions[f[:-4]]<0).any().any()), f"The submission {f} has Negative valus"
    _loaded +=1
    print(f"------> submission for model {f[:-4]} loaded.")
  except Exception as e:
    print(f"ERROR --> submission file {f} could not be loaded. Because {e}")
print(f"{_loaded} submissions loaded")
del _loaded

> Load file FINAL_submission_Naive_XGBoost_Model_None.csv
ERROR --> submission file FINAL_submission_Naive_XGBoost_Model_None.csv could not be loaded. Because 'itemID'
> Load file test_submission_TEST_Naive_XGBoost_Model_k_promo.csv
------> submission for model test_submission_TEST_Naive_XGBoost_Model_k_promo loaded.
> Load file test_submission_non_recursive_28062020.csv
------> submission for model test_submission_non_recursive_28062020 loaded.
> Load file submission_XGBoost_final_k_promo_final.csv
ERROR --> submission file submission_XGBoost_final_k_promo_final.csv could not be loaded. Because The submission submission_XGBoost_final_k_promo_final.csv has Negative valus
> Load file submission_XGBoost_final.csv
------> submission for model submission_XGBoost_final loaded.
> Load file submission_XGBoost_final_k_promo_final_corrected.csv
------> submission for model submission_XGBoost_final_k_promo_final_corrected loaded.
> Load file submission_XGBoost_final_k_promo_final_corrected_handc

In [19]:
# submissions is dictionary with filenames (without .csv) as key and corresponding dataframe as value
# print(type(submissions['test_submission_TEST_Naive_XGBoost_Model_k_promo']))

## Calculate the demand prediction (for 2 weeks) for every item and save it

In [20]:
prediction_period_end = pd.to_datetime("2018-07-13")
days = 14

prediction_period = list(map(lambda x: str(x), pd.date_range( end=prediction_period_end, periods=days)))
print(prediction_period)

index_list = list(range(1, 10464))
print(index_list[-1])

['2018-06-30 00:00:00', '2018-07-01 00:00:00', '2018-07-02 00:00:00', '2018-07-03 00:00:00', '2018-07-04 00:00:00', '2018-07-05 00:00:00', '2018-07-06 00:00:00', '2018-07-07 00:00:00', '2018-07-08 00:00:00', '2018-07-09 00:00:00', '2018-07-10 00:00:00', '2018-07-11 00:00:00', '2018-07-12 00:00:00', '2018-07-13 00:00:00']
10463


In [21]:
def preprocess_submission(team: str, submission_df, prediction_period, index_list):
  if submission_df is None:
    return
  if 'demandPrediction' not in submission_df or submission_df['demandPrediction'].isnull().values.any():
    if team is not None:
      print(f"The cummulative prediction for team {team} was not given by the team. Will calculate the demand now")

    #submission_df['demandPrediction'] = np.NaN
    submission_df['demandPrediction'] = submission_df.sum(axis=1)

  # check the prediction period
  if len(submission_df.columns.tolist()) > 1:
    range = [r for r in submission_df.columns.tolist() if r != "demandPrediction"]
    if (range != prediction_period):
      print(submission_df.columns.tolist())
      print(prediction_period)
      print("\tWrong time period!!!")
      return None
  
  # check the items
  if submission_df.index.tolist() != index_list:
    print("\tWrong items!!!")
    return None

  if len(submission_df.columns.tolist()) > 1:
    submission_df = submission_df.drop(prediction_period, axis=1)
  else:
    print("\tI hope you made sure that you submitted the prediction for the correct period. I can't check that here.")

  return submission_df


In [22]:
proc_submissions = {}

for sub_key in submissions.keys():
  print(f"Processing file {sub_key} ...")
  try:
    proc_submissions[sub_key] = preprocess_submission(sub_key, submissions[sub_key], prediction_period, index_list)
    assert len(proc_submissions[sub_key].columns) == 1, "Wrong number of columns!"
    assert proc_submissions[sub_key].columns[0] == "demandPrediction", "Column 'demandPrediction' is missing!"
    proc_submissions[sub_key].to_csv(f'{submission_path_save}/TU_Muenchen_1_{sub_key}.csv', sep='|')
    print("\tOK!")

  except Exception as e:
    print(f"\tERROR --> file {sub_key} has invalid format. {e}")
  

Processing file test_submission_TEST_Naive_XGBoost_Model_k_promo ...
['2018-06-02 00:00:00', '2018-06-03 00:00:00', '2018-06-04 00:00:00', '2018-06-05 00:00:00', '2018-06-06 00:00:00', '2018-06-07 00:00:00', '2018-06-08 00:00:00', '2018-06-09 00:00:00', '2018-06-10 00:00:00', '2018-06-11 00:00:00', '2018-06-12 00:00:00', '2018-06-13 00:00:00', '2018-06-14 00:00:00', '2018-06-15 00:00:00', 'demandPrediction']
['2018-06-30 00:00:00', '2018-07-01 00:00:00', '2018-07-02 00:00:00', '2018-07-03 00:00:00', '2018-07-04 00:00:00', '2018-07-05 00:00:00', '2018-07-06 00:00:00', '2018-07-07 00:00:00', '2018-07-08 00:00:00', '2018-07-09 00:00:00', '2018-07-10 00:00:00', '2018-07-11 00:00:00', '2018-07-12 00:00:00', '2018-07-13 00:00:00']
	Wrong time period!!!
	ERROR --> file test_submission_TEST_Naive_XGBoost_Model_k_promo has invalid format. 'NoneType' object has no attribute 'columns'
Processing file test_submission_non_recursive_28062020 ...
	I hope you made sure that you submitted the predictio